In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from gensim.models import Word2Vec
import gensim.models.keyedvectors as word2vec
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import keras
from IPython.display import SVG
from keras import layers, activations, optimizers
from tensorflow.python.keras.layers import Lambda 
import re
from keras.layers import advanced_activations
from random import randrange
import pickle
import matplotlib.pyplot as plt
from keras import backend as K
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV

Using TensorFlow backend.


In [0]:

def read_data(filename):
    file = []
    with open(filename,encoding="utf8") as fp:
        line = fp.readline()
#         line = gs.utils.simple_preprocess(line)
        while line:
           file.append(line)
           line = fp.readline()
#            line = gs.utils.simple_preprocess(line)
    file = np.reshape(file, (len(file),1))
    fp.close()
    return file

def load_train_data(filepath, minidx, maxidx):
    filename = filepath + str(minidx) +'.txt'
    file = read_data(filename)
    data = np.array(file)
    for i in range(minidx +1 , maxidx +1 ):
        filename = filepath + str(i) +'.txt'
        file = read_data(filename)
        data = np.concatenate((data,file), axis=0)

    labels = []
    for i in range(minidx, maxidx +1):
        labels += [i]*2000

    return data, labels


def load_test_data(filepath):
    filename_1 = filepath +  'data.txt'
    file = read_data(filename_1)
    data = np.array(file)

    filename_2 = filepath + 'label.txt'
    with open(filename_2) as f:
        labels = []
        for line in f:
            labels.append(int(line))

    f.close()
    labels = np.array(labels)
    return data, labels
  
def one_hot_coding(Y,num_class):
    Y_one_hot_code = np.zeros((len(Y),num_class))
    for i in np.arange(len(Y)):
        Y_one_hot_code[i][Y[i]-1] = 1
    return Y_one_hot_code    
  
def pre_process(text):
    text = text.lower()
    # remove tags
    text = re.sub("<!--?.*?-->", "", text)
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    text = text.split()
    return text

def get_split_word(data):
  doc_list = []
  for d in data:
    doc_list.append(pre_process(d[0]))
  return doc_list

def get_doc_matrix(model, data, max_len,i_prediction, h,negative_size, num_features):
  matrix_input_1 = []
  matrix_input_2 = []
  words = []

  for word in model.vocab.keys():
    words.append(word)

  for d in data:
    matrix_1 = np.zeros((i_prediction, num_features))
    matrix_2 = np.zeros((h + negative_size, num_features ))

    for i in np.arange(i_prediction):
      if(i < len(d)):
        matrix_1[i] = model[d[i]]
      # else:
        # matrix_1[i] = np.zeros((num_features))
    matrix_input_1.append(matrix_1)

    for i in np.arange(h):
      if(i + i_prediction < len(d)):
        matrix_2[i] = model[d[i + i_prediction]]

    i = 0 
    while(i < negative_size):
      idx = randrange(len(words))
      if(words[idx] not in d):
         matrix_2[i + h] = model[words[idx]]
         i += 1
      else: continue
    matrix_input_2.append(matrix_2)
  matrix_input_2 = np.array(matrix_input_2)
  matrix_input_1 = np.array(matrix_input_1)

  return matrix_input_1, matrix_input_2
    
def get_doc_embedding(model,train_model, data, max_len, num_features, tags):
  if(tags == "test"):
    data = get_split_word(data)
    h =5 
    data, data1 = get_doc_matrix(model, data, max_len, i_prediction, h , negative_size, num_features)
    data = data.reshape(data.shape[0], i_prediction, num_features  ,1)
    print("ok")
    print(data.shape)

  print(data.shape)

  # doc1 = [pre_process(doc1)]
  # doc2 = [pre_process(doc2)]
  # d1 = get_doc_matrix(doc1,model, max_len, num_features)
  # d1 = d1.reshape(d1.shape[0], max_len, num_features,1)

  # d2 = get_doc_matrix(doc2, model, max_len, num_features)
  # d2 = d2.reshape(d2.shape[0], max_len, num_features,1)
  get_embedding_layer_output = K.function([train_model.layers[0].input],
                                  [train_model.layers[7].output])
  
  embedding_doc = get_embedding_layer_output([data])[0]
  # embedding_d2 = get_embedding_layer_output([d2])[0]
  # print(embedding_d1)
  # print(embedding_d2)
  return embedding_doc




    



In [0]:
num_filter_1 = 64
filter_size_1_h = 32 
filter_size_1_w = 32
num_filter_2 = 128
filter_size_2_h = 4
filter_size_2_w = 96


def train_model_new(file_model,X, y, max_len ,h, num_features ,negative_size):
  input_shape = (max_len, num_features, 1)
  input_x = layers.Input(shape=input_shape)
  print(input_x.get_shape)

  Conv1 = layers.Convolution2D(num_filter_1,( filter_size_1_h, filter_size_1_w), 
                                 strides= (1,1))(input_x)
  print(Conv1.get_shape)

  Conv2 =  layers.Convolution2D(num_filter_1,(filter_size_1_h, filter_size_1_w), 
                                 strides= (1,1), activation = 'sigmoid' )(input_x)
  print(Conv2.get_shape)

  GLU_layer_1 = layers.multiply([Conv1,Conv2])

  Conv3 = layers.Convolution2D(num_filter_2,( filter_size_2_h, filter_size_2_w), 
                                 strides= (1,1))(GLU_layer_1)
  print(Conv3.get_shape)

  Conv4 =  layers.Convolution2D(num_filter_2,(filter_size_2_h, filter_size_2_w), 
                                 strides= (1,1), activation = 'sigmoid' )(GLU_layer_1)
  print(Conv4.get_shape)

  GLU_layer_2 = layers.multiply([Conv3,Conv4])
  print(GLU_layer_2.get_shape)
  Max_pool = layers.MaxPool2D((3,1))(GLU_layer_2)
  print(Max_pool.get_shape)
  # Global_pool = layers.GlobalAveragePooling2D()(Max_pool)
  # print(Global_pool.get_shape)
  flatten = layers.Flatten()(Max_pool)
  # print(flatten.get_shape)
  output_embedding = layers.Dense(128)(flatten)
  print(output_embedding.get_shape)

  input_y = ( h + negative_size, num_features)
  input_y = keras.Input(shape=input_y)
  print(input_y.get_shape)

  output = layers.dot([input_y, output_embedding], axes= [2,1])
  print(output.get_shape)
  # output = activations.sigmoid(output)
  # output = Lambda(sigmoid_layer)(output)
  # output = layers.Dense(15,activation ='sigmoid')(output)
  # print(output.value)
  output = layers.Lambda(lambda z: activations.sigmoid(z))(output)
  model = keras.Model(inputs = [input_x, input_y], outputs = output)

  yTrain = np.concatenate([np.ones((len(X), h)), np.zeros((len(X), negative_size))],axis =1)
  # yTrain = np.ones((len(X),max_len))

  optimization = optimizers.SGD(lr = 0.01)
  model.compile(loss=  'mean_squared_error', #binary_crossentropy',
                  optimizer= optimization, metrics = ['accuracy'])
  history = model.fit([X, y], yTrain, epochs=10, batch_size=100, validation_split= 0.2)
  print("ok")

  model.save(file_model +'train_model.model')
  return model, history



In [5]:
if __name__ == '__main__':
  path = '/content/drive/My Drive/Intership/'
  num_features = 128
  i_prediction = 400
  h = 10
  negative_size = 10
  max_len = 25
  data_train, labels_train = load_train_data(path + 'classify_data/train/', 1, 13)
  data_test, labels_test = load_test_data(path +'classify_data/test/')
  model = word2vec.KeyedVectors.load(path + 'Week2/CNN/model/preprocessing_2.h5')
  data_train = get_split_word(data_train)
  matrix_input_1, matrix_input_2 = get_doc_matrix(model, data_train, max_len, i_prediction, h , negative_size, num_features)
  print(matrix_input_1.shape)
  print(matrix_input_2.shape)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(26000, 400, 128)
(26000, 20, 128)


In [0]:
l = []
for d in data_train:
  l.append(len(d))

print(np.max(l))

4472


In [0]:
import matplotlib.pyplot as plt
plt.hist(l, bins = np.arange(0, 4500, 50))

In [0]:
matrix_input_1 = matrix_input_1.reshape(matrix_input_1.shape[0], i_prediction, num_features  ,1)
matrix_input_2 = matrix_input_2.reshape(matrix_input_2.shape[0],h + negative_size, 128)

In [0]:
file_model = path + 'Week2/CNN/model/train_model_2.model'
train_model, history = train_model_new(file_model, matrix_input_1, matrix_input_2, i_prediction,h, num_features, negative_size)

W0809 10:43:11.395848 140680067069824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 10:43:11.439749 140680067069824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 10:43:11.447734 140680067069824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 10:43:11.514735 140680067069824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



<bound method Tensor.get_shape of <tf.Tensor 'input_1:0' shape=(?, 400, 128, 1) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'conv2d_1/BiasAdd:0' shape=(?, 369, 97, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'conv2d_2/Sigmoid:0' shape=(?, 369, 97, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'conv2d_3/BiasAdd:0' shape=(?, 366, 2, 128) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'conv2d_4/Sigmoid:0' shape=(?, 366, 2, 128) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'multiply_2/mul:0' shape=(?, 366, 2, 128) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'max_pooling2d_1/MaxPool:0' shape=(?, 122, 2, 128) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 128) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'input_2:0' shape=(?, 20, 128) dtype=float32>>


W0809 10:43:11.980163 140680067069824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0809 10:43:12.161157 140680067069824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



<bound method Tensor.get_shape of <tf.Tensor 'dot_1/Squeeze:0' shape=(?, 20) dtype=float32>>


W0809 10:43:12.202748 140680067069824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 20800 samples, validate on 5200 samples
Epoch 1/10
20800/20800 [==============================] - 470s 23ms/step - loss: 0.2039 - acc: 0.6385 - val_loss: 0.1399 - val_acc: 0.5110
Epoch 2/10
20800/20800 [==============================] - 436s 21ms/step - loss: 0.1327 - acc: 0.6156 - val_loss: 0.1130 - val_acc: 0.4988
Epoch 3/10
20800/20800 [==============================] - 436s 21ms/step - loss: 0.1207 - acc: 0.6254 - val_loss: 0.1081 - val_acc: 0.5150
Epoch 4/10
20800/20800 [==============================] - 436s 21ms/step - loss: 0.1170 - acc: 0.6285 - val_loss: 0.1062 - val_acc: 0.5212
Epoch 5/10
20800/20800 [==============================] - 436s 21ms/step - loss: 0.1152 - acc: 0.6318 - val_loss: 0.1053 - val_acc: 0.5254
Epoch 6/10
20800/20800 [==============================] - 436s 21ms/step - loss: 0.1141 - acc: 0.6345 - val_loss: 0.1048 - val_acc: 0.5254
Epoch 7/10
20800/20800 [==============================] - 436s 21ms/step - loss: 0.1134 - acc: 0.6361 - val_loss: 0.1

In [0]:
from keras.models import load_model
file_model = path + 'Week2/CNN/model/train_model_2.model'

train_model = load_model(file_model)

del matrix_input_2


In [8]:
doc_embedding_matrix = get_doc_embedding(model,train_model, matrix_input_1, i_prediction, num_features, tags = "train")
print(doc_embedding_matrix.shape)
del matrix_input_1

(26000, 400, 128, 1)
(26000, 128)


In [9]:
doc_embedding_matrix_test = get_doc_embedding(model,train_model, data_test, i_prediction, num_features, tags = "test")
print(doc_embedding_matrix_test.shape)
del data_test

ok
(6500, 400, 128, 1)
(6500, 400, 128, 1)
(6500, 128)


In [0]:
print(labels_train)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [12]:
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(doc_embedding_matrix, labels_train)
print('Best score for training data:', svm_model.best_score_,"\n") 
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

Best score for training data: 0.5805769230769231 

Best C: 1000 

Best Kernel: rbf 

Best Gamma: 0.001 



In [12]:

num_class = 13
# y_train = one_hot_coding(labels_train, num_class)
# t_test = one_hot_coding(labels_test, num_class)
# y_train = np.array(y_train)
# y_test   = np.array(y_test)
#     labels_train = one_hot_coding(labels_train, num_class)
#     labels_test  = one_hot_coding(labels_test, num_class)
svm_model = SVC(kernel="linear", C = 1000).fit(doc_embedding_matrix, labels_train)
score = svm_model.score(doc_embedding_matrix_test, labels_test)
print("\n %.2f%%" % (score*100))


 54.52%


In [16]:
num_class = 13
# y_train = one_hot_coding(labels_train, num_class)
# t_test = one_hot_coding(labels_test, num_class)
# y_train = np.array(y_train)
# y_test   = np.array(y_test)
#     labels_train = one_hot_coding(labels_train, num_class)
#     labels_test  = one_hot_coding(labels_test, num_class)
svm_model = SVC(kernel="rbf", gamma = 0.001, C = 100).fit(doc_embedding_matrix, labels_train)
score = svm_model.score(doc_embedding_matrix_test, labels_test)
print("\n %.2f%%" % (score*100))


 40.77%
